### We're gonna do some preprocessing

I am doing this on Casper because I cannot figure out Asha, however I think the directory structure is the same once we navigate to the directory so maybe just start with

In [1]:
machine = 'casper' # 'casper' or 'asha'

I am going to write code that can load in data for a specific model and variable from the training. Maybe a function that can be called to do that.

In [2]:
# I/O / data wrangling
import glob
import numpy as np
import xarray as xr
import gc
import preprocessing
import importlib as imp

<module 'preprocessing' from '/glade/u/home/egordon/ForceSMIP/preprocessing.py'>

In [3]:
if machine == 'casper':
    root_dir = "/glade/campaign/cgd/cas/asphilli/ForceSMIP/"
elif machine == 'asha':
    root_dir = "/barnes-scratch/DATA/ForceSMIP/"
    
cmipTable = {
    "pr": "Amon",
    "psl": "Amon",
    "tas": "Amon",
    "zmta": "Amon",
    "tos": "Omon",
    "siconc": "OImon",
    "monmaxpr": "Aday",
    "monmaxtasmax": "Aday",
    "monmintasmin": "Aday",
}
cmipVar = {
    "pr": "pr",
    "psl": "psl",
    "tas": "tas",
    "zmta": "ta",
    "tos": "tos",
    "siconc": "siconc",
    "monmaxpr": "pr",
    "monmaxtasmax": "tasmax",
    "monmintasmin": "tasmin",
}
evalPeriods = {
    "Tier1": ("1950-01-01", "2022-12-31"),
    "Tier2": ("1900-01-01", "2022-12-31"),
    "Tier3": ("1979-01-01", "2022-12-31"),
}

fullmems = {
    "MIROC6": 50,
    "CESM2": 50,
    "CanESM5": 25,
    "MPI-ESM1-2-LR": 30,
    "MIROC-ES2L": 30,
}

In [46]:
imp.reload(preprocessing)

X_eval = preprocessing.make_eval_mem()
X_eval.shape

(73, 72, 144)

In [22]:
Xpr,Ypr_f,Ypr_i = preprocessing.make_data(models=["CanESM5"],var="pr",mems=np.arange(5))
Xtos,_,_ = preprocessing.make_data(models=["CanESM5"],var="tos",mems=np.arange(5))

In [23]:
Xpr.shape

(365, 72, 144)

In [24]:
Xtos.shape

(365, 72, 144)

In [29]:
X_all = np.stack((Xpr,Xtos),axis=-1)
X_all.shape

(365, 72, 144, 2)

In [50]:
imp.reload(preprocessing)
Xpr,Ypr_f,Ypr_i = preprocessing.make_data(models=["CanESM5","CESM2"],var="pr",mems=np.arange(5))
Xtos,_,_ = preprocessing.make_data(models=["CanESM5","CESM2"],var="tos",mems=np.arange(5))

Xall = preprocessing.stack_variable((Xpr,Xtos))
Xall.shape

CanESM5
CESM2
CanESM5
CESM2


(730, 72, 144, 2)

In [47]:
evalmem = "1H"
var = "pr"
timecut = "Tier1"

filelist = root_dir + "Evaluation-"+timecut+"/" + cmipTable[var] + "/" + var + "/" + var + "*" + evalmem + "*.nc"
file = glob.glob(filelist)[0]
ds = xr.open_dataset(file)

In [49]:
ds.pr

<xarray.DataArray 'pr' (time: 876, lat: 72, lon: 144)>
[9082368 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 1950-01-16T12:00:00 ... 2022-12-16T12:00:00
  * lat      (lat) float64 -88.75 -86.25 -83.75 -81.25 ... 83.75 86.25 88.75
  * lon      (lon) float64 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
Attributes:
    long_name:  Precipitation
    units:      mm/day

In [ ]:
arr_name = os.path.join('saved_models', exp_name, exp_name+str(experiment_settings["seed"])+"_preds.npz")
np.savez(arr_name,Ptrain=Ptrain,
                  Pval=Pval,
                  Ptest=Ptest,
                  Ptrain_us=Ptrain_us,
                  Pval_us=Ptest_us,
                  Pval_us=Ptest_us,
                  )

